# Agentes Inteligentes #

Este notebook serve como material de apoio para os tópicos abordados no **Capítulo 2 - Agentes Inteligentes** do livro *Artificial Intelligence: A Modern Approach*. Este notebook utiliza implementações do módulo [agents.py](https://github.com/aimacode/aima-python/blob/master/agents.py). Vamos começar importando tudo do módulo de agentes.

In [ ]:
#Pode ser necessário instalar algumas libs
#!pip install ipythonblocks
#!pip install qpsolvers



In [ ]:
from agents import *
from notebook import psource

## CONTENTS

* Overview
* Agent
* Environment
* Simple Agent and Environment
* Agents in a 2-D Environment
* Wumpus Environment

## OVERVIEW

Um agente, conforme definido em 2.1, é qualquer coisa que possa perceber seu ambiente através de sensores e agir sobre esse ambiente através de atuadores, com base em seu programa de agente. Isso pode ser um cachorro, um robô ou até mesmo você. Contanto que você possa perceber o ambiente e agir sobre ele, você é um agente. Este notebook explicará como implementar um agente simples, criar um ambiente e implementar um programa que ajude o agente a agir sobre o ambiente com base em suas percepções.

## AGENT

Let us now see how we define an agent. Run the next cell to see how `Agent` is defined in agents module.

In [ ]:
psource(Agent)

A classe `Agent` possui dois métodos:
* `__init__(self, program=None)`: O construtor define vários atributos do Agente. Estes incluem:

    * `alive`: que acompanha se o agente está vivo ou não
    
    * `bump`:  que rastreia se o agente colide com uma borda do ambiente (por exemplo, uma parede em um parque)
    
    * `holding`: que é uma lista contendo os Things (objetos) que um agente está segurando
    
    * `performance`:que avalia as métricas de desempenho do agente
    
    * `program`: que é o programa do agente e mapeia as percepções do agente para ações no ambiente. Se nenhuma implementação for fornecida, o padrão é solicitar ao usuário que forneça ações para cada percepção.
    
* `can_grab(self, thing)`:  É usado quando um ambiente contém objetos que um agente pode pegar e carregar. Por padrão, um agente não pode carregar nada.


## ENVIRONMENT
Now, let us see how environments are defined. Running the next cell will display an implementation of the abstract `Environment` class.

In [ ]:
psource(Environment)

A classe `Environment` possui muitos métodos! Mas a maioria deles é incrivelmente simples, então vamos ver os que utilizaremos neste notebook.

* `thing_classes(self)`: Retorna um array estático de sub-classes de `Thing` que determinam quais coisas são permitidas no ambiente e quais não são.

* `add_thing(self, thing, location=None)`: Adiciona uma coisa ao ambiente em uma determinada localização.

* `run(self, steps)`: Executa um ambiente com o agente nele por um determinado número de passos.

* `is_done(self)`: Retorna verdadeiro se o objetivo do agente e do ambiente foi concluído.

As próximas duas funções devem ser implementadas por cada subclasse de `Environment` para que o agente receba percepções e execute ações.

* `percept(self, agent)`: Dado um agente, este método retorna uma lista de percepções que o agente observa no momento atual.

* `execute_action(self, agent, action)`: O ambiente reage a uma ação realizada por um determinado agente. As mudanças podem resultar em o agente experimentar novas percepções ou outros elementos reagirem à entrada do agente.

## AGENTE SIMPLES E AMBIENTE

"Vamos começar utilizando a classe `Agent` para criar nosso primeiro agente - um cachorro cego (a blind dog)."


In [ ]:
class BlindDog(Agent):
    def eat(self, thing):
        print("Dog: Ate food at {}.".format(self.location))
            
    def drink(self, thing):
        print("Dog: Drank water at {}.".format( self.location))

dog = BlindDog()

### Erro: Não foi possível encontrar um programa válido para BlindDog, voltando ao padrão.
O que acabamos de fazer é criar um cachorro que só pode sentir o que está em sua localização (já que ele é cego) e pode comer ou beber. Vamos ver se ele está vivo...

In [ ]:
print(dog.alive)

![Cool dog](https://gifgun.files.wordpress.com/2015/07/wpid-wp-1435860392895.gif)
Este é o nosso cachorro. Vamos considerar que ele está com fome e precisa sair em busca de comida. Para que ele faça isso, precisamos dar a ele um **programa**. Mas antes disso, vamos criar um **parque** para o nosso cachorro brincar.

### Ambiente (ENVIRONMENT) - Parque (Park)

Um parque é um exemplo de ambiente porque nosso cachorro pode percebê-lo e agir sobre ele. A classe <b>Environment</b> é uma classe abstrata, então teremos que criar nossa própria subclasse a partir dela antes de podermos usá-la.

In [ ]:
class Food(Thing):
    pass

class Water(Thing):
    pass

class Park(Environment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.

    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles


### PROGRAM - BlindDog
Agora que temos uma classe <b>Park</b>, reimplementamos nosso <b>BlindDog</b> para que ele possa se mover para baixo e comer comida ou beber água apenas se estiver presente.

In [ ]:
class BlindDog(Agent):
    location = 1
    
    def movedown(self):
        self.location += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

Agora é hora de implementar um módulo de <b>program</b> para o nosso cachorro. Um programa controla como o cachorro age sobre seu ambiente. Nosso programa será bem simples, como mostrado na tabela abaixo.
<table>
    <tr>
        <td><b>Percept:</b> </td>
        <td>Feel Food </td>
        <td>Feel Water</td>
        <td>Feel Nothing</td>
   </tr>
   <tr>
       <td><b>Action:</b> </td>
       <td>eat</td>
       <td>drink</td>
       <td>move down</td>
   </tr>
        
</table>

In [ ]:
def program(percepts):
    '''Returns an action based on the dog's percepts'''
    for p in percepts:
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
    return 'move down'

Agora vamos rodar nossa simulação criando um parque com comida, água e o nosso cachorro.

In [ ]:
park = Park()
dog = BlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, 1)
park.add_thing(dogfood, 5)
park.add_thing(water, 7)

park.run(15)

Notice that the dog moved from location 1 to 4, over 4 steps, and ate food at location 5 in the 5th step.

Let's continue this simulation for 5 more steps.

In [ ]:
park.run(5)

Perfect! Note how the simulation stopped after the dog drank the water - exhausting all the food and water ends our simulation, as we had defined before. Let's add some more water and see if our dog can reach it.

In [ ]:
park.add_thing(water, 15)
park.run(10)

Acima, aprendemos a implementar um agente, seu programa e um ambiente no qual ele atua. No entanto, este foi um caso muito simples. Vamos tentar adicionar complexidade criando um ambiente bidimensional!

## AGENTES EM UM AMBIENTE 2D

Para evitar ler tantos logs sobre o que nosso cachorro fez, vamos adicionar um pouco de gráficos enquanto tornamos nosso Parque 2D. Para isso, precisaremos torná-lo uma subclasse de <b>GraphicEnvironment</b> em vez de Environment. Parques implementados como subclasses da classe <b>GraphicEnvironment</b> adicionam essas propriedades extras:

- Nosso parque é indexado no 4º quadrante do plano X-Y.
- Toda vez que criamos um parque como subclasse de <b>GraphicEnvironment</b>, precisamos definir as cores de todos os itens que planejamos colocar no parque. As cores são definidas no típico [<b>formato RGB digital de 8 bits</b>](https://en.wikipedia.org/wiki/RGB_color_model#Numeric_representations), comum na web.
- Cercas são adicionadas automaticamente a todos os parques para que nosso cachorro não saia dos limites do parque — simplesmente não é seguro para cachorros cegos ficarem fora do parque sozinhos! <b>GraphicEnvironment</b> fornece a função `is_inbounds` para verificar se nosso cachorro tenta sair do parque.

Primeiro, vamos tentar atualizar nosso ambiente `Park` unidimensional apenas substituindo sua superclasse por `GraphicEnvironment`.


In [ ]:
class Park2D(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

class BlindDog(Agent):
    location = [0,1] # change location to a 2d value
    direction = Direction("down") # variable to store the direction our dog is facing
    
    def movedown(self):
        self.location[1] += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

Agora vamos testar este novo parque com o mesmo cachorro, comida e água. Colorimos nosso cachorro com vermelho e marcamos a comida e a água com laranja e azul, respectivamente

In [ ]:
park = Park2D(5,20, color={'BlindDog': (200,0,0), 'Food': (230, 115, 40),'Water': (0, 200, 200)}) 
# park largura = 5 (width) e altura = 20 (height)
dog = BlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, [0,1])
park.add_thing(dogfood, [0,5])
park.add_thing(water, [0,7])
morewater = Water()
park.add_thing(morewater, [0,15])
print("O BlindDog começa em (0,1) voltado para baixo, vamos ver se ele consegue encontrar alguma comida!")
park.run(20)

Vemos imediatamente que o código funciona, mas nosso cachorro cego **não faz uso** do espaço bidimensional disponível para ele. Vamos deixar nosso cachorro mais enérgico para que ele vire e avance, em vez de sempre se mover para baixo. Ao fazer isso, também precisaremos fazer algumas alterações em nosso ambiente para lidar com esse movimento extra.

### PROGRAMA - EnergeticBlindDog

Vamos fazer nosso cachorro virar ou avançar aleatoriamente — exceto quando ele estiver na borda do nosso parque — nesse caso, faremos ele mudar de direção explicitamente, virando para evitar tentar sair do parque. No entanto, nosso cachorro é cego, então ele não saberia para que lado virar — ele teria que tentar aleatoriamente.

<table>
    <tr>
        <td><b>Percepto:</b> </td>
        <td>Sente Comida </td>
        <td>Sente Água</td>
        <td>Não Sente Nada</td>
   </tr>
   <tr>
       <td><b>Ação:</b> </td>
       <td>comer</td>
       <td>beber</td>
       <td>
       <table>
           <tr>
               <td><b>Lembrar de estar na borda:</b></td>
               <td>Na Borda</td>
               <td>Não na Borda</td>
           </tr>
           <tr>
               <td><b>Ação:</b></td>
               <td>Virar à Esquerda / Virar à Direita <br> (50% - 50% de chance)</td>
               <td>Virar à Esquerda / Virar à Direita / Avançar <br> (25% - 25% - 50% de chance)</td>
           </tr>
       </table>
       </td>
   </tr>
</table>

In [ ]:
from random import choice

class EnergeticBlindDog(Agent):
    location = [0,1]
    direction = Direction("down")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location). 
           Avançar só é possível se for bem-sucedido (ou seja, se o destino for uma localização válida)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False
        
def program(percepts):
    '''Returns an action based on it's percepts'''
        
    for p in percepts: # first eat or drink - you're a dog!
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
        if isinstance(p,Bump): # então verifique se você está na borda e precisa virar
            # Bump: rastreia se o agente colide com uma borda do ambiente
            turn = False
            choice = random.choice((1,2));
        else:
            choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward
    if choice == 1:
        return 'turnright'
    elif choice == 2:
        return 'turnleft'
    else:
        return 'moveforward'
    

### AMBIENTE (ENVIRONMENT) - Park2D

Também precisamos modificar nosso parque adequadamente, para ser capaz de lidar com todas as novas ações que nosso cachorro deseja executar. Além disso, precisaremos impedir que nosso cachorro se mova para locais além dos limites do parque — simplesmente não é seguro para cachorros cegos ficarem fora do parque sozinhos.

In [ ]:
class Park2D(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        loc = copy.deepcopy(agent.location) # find out the target location (descobrir a localização de destino)
        #Check if agent is about to bump into a wall (Verifique se o agente está prestes a esbarrar em uma parede)
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        if not self.is_inbounds(loc):# Verifica se nosso cachorro tenta sair do parque.
            things.append(Bump())
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            print('{} decidiu ir para {} na localização: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decidiu ir para {} na localização: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            print('{} decidiu mover {} direcionado para a localização: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]):
                    print('{} comeu {} na localização: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]):
                    print('{} bebeu {} na localização: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles


Agora que nosso parque está pronto para o movimento 2D do nosso cachorro, vamos testá-lo!

In [ ]:
park = Park2D(5,5, color={'EnergeticBlindDog': (200,0,0), 'Water': (0, 200, 200), 'Food': (230, 115, 40)})
dog = EnergeticBlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, [0,0])
park.add_thing(dogfood, [1,2])
park.add_thing(water, [0,1])
morewater = Water()
morefood = Food()
park.add_thing(morewater, [2,4])
park.add_thing(morefood, [4,3])
print("O cachorro começou em [0,0], voltado para baixo. Vamos ver se ele encontrou alguma comida ou água!")
park.run(50)

In [ ]:
#Quadrado vermelho é o AGENTE caçando comida e agua

## Aula Prática: Implementando um Agente Inteligente em Python

### Objetivo
Nesta aula, vamos implementar um **agente inteligente** que aprende a interagir com um ambiente usando a biblioteca `gym`. O ambiente escolhido será o **CartPole**, um problema clássico de controle onde o objetivo é equilibrar uma vara em um carrinho.


### Pré-requisitos
1. **Python 3** instalado.
2. Bibliotecas necessárias:
   - `gym`: Para criar o ambiente.
   - `numpy`: Para cálculos numéricos.
   - `matplotlib`: Para visualização dos resultados.

In [5]:
pip install gymnasium numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 8.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


### Passo 1: Importando as Bibliotecas

In [6]:
import gymnasium as gym
import numpy as np

### Passo 2: Criando o Ambiente
Vamos criar o ambiente **CartPole**:

In [7]:
env = gym.make("CartPole-v1")


- **O que é o CartPole?**
  - O ambiente consiste em um carrinho que pode se mover para a esquerda ou direita.
  - Uma vara é presa ao carrinho, e o objetivo é equilibrar a vara por quanto tempo for possível.

### 3. Loop de Interação com o Ambiente
O agente segue uma política simples (aleatória) para interagir com o ambiente.

In [8]:
# Inicializando variáveis
num_episodios = 5

for episodio in range(num_episodios):
    estado = env.reset()  # Reinicia o ambiente
    done = False
    score = 0

    while not done:
        env.render()  # Renderiza o ambiente
        
        # Escolha da ação (aleatória neste exemplo)
        acao = env.action_space.sample()
        
        # Executa a ação
        novo_estado, recompensa, done, _, _ = env.step(acao)
        
        # Acumula a pontuação
        score += recompensa

    print(f"Episódio {episodio + 1}: Pontuação = {score}")

env.close()

Episódio 1: Pontuação = 20.0
Episódio 2: Pontuação = 13.0
Episódio 3: Pontuação = 35.0
Episódio 4: Pontuação = 46.0
Episódio 5: Pontuação = 18.0


/home/eltonss/anaconda3/lib/python3.11/site-packages/gymnasium/envs/classic_control/cartpole.py:250: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(


## Extensão: Introduzindo Inteligência ao Agente
Podemos adicionar um agente mais avançado usando técnicas de aprendizado por reforço. Aqui, implementamos uma política simples que ajusta suas ações com base no estado percebido.


In [14]:
class AgenteSimples:
    def __init__(self, action_space):
        self.action_space = action_space

    def escolher_acao(self, estado):
        # Exemplo: Política baseada no estado (aqui, aleatória para simplicidade)
        return self.action_space.sample()

# Criando o agente
agente = AgenteSimples(env.action_space)

# Reexecutando o loop com o agente
for episodio in range(num_episodios):
    estado = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        acao = agente.escolher_acao(estado)
        novo_estado, recompensa, done, _, _ = env.step(acao)
        score += recompensa

    print(f"Episódio {episodio + 1}: Pontuação = {score}")

env.close()

Episódio 1: Pontuação = 12.0
Episódio 2: Pontuação = 12.0
Episódio 3: Pontuação = 26.0
Episódio 4: Pontuação = 16.0
Episódio 5: Pontuação = 13.0


# Implementação do Ambiente Aspirador de Pó com Gymnasium


### Descrição
O ambiente consiste em:
- Duas salas (esquerda e direita) que podem estar limpas ou sujas.
- O agente pode:
  - Mover-se entre as salas.
  - Limpar a sala atual.

---


### Código do Ambiente


In [3]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class AspiradorEnv(gym.Env):
    def __init__(self):
        super(AspiradorEnv, self).__init__()
        # Espaço de ação: 0 = direita, 1 = esquerda, 2 = limpar
        self.action_space = spaces.Discrete(3)
        # Espaço de observação: [estado da sala esquerda, estado da sala direita, posição do agente]
        self.observation_space = spaces.MultiDiscrete([2, 2, 2])
        self.reset()

    def reset(self):
        # Estado inicial: ambas as salas sujas e agente na sala esquerda
        self.state = np.array([1, 1, 0])
        return self.state

    def step(self, action):
        recompensa = 0

        # Mover para a direita
        if action == 0 and self.state[2] == 0:
            self.state[2] = 1
        # Mover para a esquerda
        elif action == 1 and self.state[2] == 1:
            self.state[2] = 0
        # Limpar a sala atual
        elif action == 2:
            if self.state[self.state[2]] == 1:
                self.state[self.state[2]] = 0
                recompensa = 1

        # O episódio termina quando ambas as salas estão limpas
        done = np.all(self.state[:2] == 0)

        return self.state, recompensa, done, {}

    def render(self):
        sala = ["Limpo" if x == 0 else "Sujo" for x in self.state[:2]]
        posicao = "Esquerda" if self.state[2] == 0 else "Direita"
        print(f"Sala Esquerda: {sala[0]}, Sala Direita: {sala[1]}, Posição do Agente: {posicao}")

## Implementação do Agente

### Código do Agente

In [4]:
class AgenteSimples:
    def __init__(self):
        pass

    def escolher_acao(self, estado):
        # Se a sala atual estiver suja, limpa
        if estado[estado[2]] == 1:
            return 2  # Limpar
        # Caso contrário, move para a próxima sala
        elif estado[2] == 0:
            return 0  # Mover para a direita
        else:
            return 1  # Mover para a esquerda

## Simulação

### Código da Simulação

In [5]:
# Inicializando o ambiente e o agente
ambiente = AspiradorEnv()
agente = AgenteSimples()

num_episodios = 5

for episodio in range(num_episodios):
    estado = ambiente.reset()
    done = False
    passos = 0

    print(f"\nEpisódio {episodio + 1}:")
    while not done:
        ambiente.render()
        acao = agente.escolher_acao(estado)
        estado, recompensa, done, _ = ambiente.step(acao)
        passos += 1
        print(f"Ação: {acao}, Recompensa: {recompensa}")

    print(f"Episódio concluído em {passos} passos!")


Episódio 1:
Sala Esquerda: Sujo, Sala Direita: Sujo, Posição do Agente: Esquerda
Ação: 2, Recompensa: 1
Sala Esquerda: Limpo, Sala Direita: Sujo, Posição do Agente: Esquerda
Ação: 0, Recompensa: 0
Sala Esquerda: Limpo, Sala Direita: Sujo, Posição do Agente: Direita
Ação: 2, Recompensa: 1
Episódio concluído em 3 passos!

Episódio 2:
Sala Esquerda: Sujo, Sala Direita: Sujo, Posição do Agente: Esquerda
Ação: 2, Recompensa: 1
Sala Esquerda: Limpo, Sala Direita: Sujo, Posição do Agente: Esquerda
Ação: 0, Recompensa: 0
Sala Esquerda: Limpo, Sala Direita: Sujo, Posição do Agente: Direita
Ação: 2, Recompensa: 1
Episódio concluído em 3 passos!

Episódio 3:
Sala Esquerda: Sujo, Sala Direita: Sujo, Posição do Agente: Esquerda
Ação: 2, Recompensa: 1
Sala Esquerda: Limpo, Sala Direita: Sujo, Posição do Agente: Esquerda
Ação: 0, Recompensa: 0
Sala Esquerda: Limpo, Sala Direita: Sujo, Posição do Agente: Direita
Ação: 2, Recompensa: 1
Episódio concluído em 3 passos!

Episódio 4:
Sala Esquerda: Sujo, 